In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Jul 29 09:17:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    31W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/Vela-Yang/vela_SR.git SR

fatal: destination path 'SR' already exists and is not an empty directory.


In [4]:
import torch
from SR.models import AVS3Filter
from SR.dataset import VelaDataset
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
from tqdm.notebook import tqdm


# 是否有GPU加速
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    num_workers = 16
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False



In [5]:
# 加载网络模型
# load the network
net_filename = './SR/Trained_Networks/256.pkl'
model = torch.load(net_filename, map_location=device)

In [ ]:
def clear_overflow(tensor):
  zero_value = torch.tensor([0]).to(device, torch.float)
  neg_clear = torch.heaviside(tensor, zero_value)
  result_tensor = tensor * neg_clear + 0.01
  overflow = result_tensor - 255
  pos_clear = torch.heaviside(overflow, zero_value) * overflow
  result_tensor = result_tensor - pos_clear
  result_tensor = result_tensor.floor()
  result_tensor = result_tensor.squeeze(0)
  result_tensor = result_tensor.to("cpu", torch.uint8)
  return result_tensor

In [ ]:

from torchvision.io import read_image, ImageReadMode, write_jpeg
torch.cuda.empty_cache()
# 输入图像
im = read_image('1.jpg', mode=ImageReadMode.GRAY)
im = im.to(device, torch.float) / 255
# 网络输出
out = model(im.unsqueeze(0)) * 255

out = clear_overflow(out)
write_jpeg(out.cpu(), '2.jpg')